# Toronto neighborhoods reading
We start by importing needed libraries

In [1]:
import requests
import pandas as pd
import numpy as np

Then I link the address of the page and read it pandas. Finally I drop all the lines with Postal Codes nont assigned.

In [2]:
page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(page)
df = pd.read_html(response.text)
df=df[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
df2 = df[df.Borough != 'Not assigned']
df2['Postal Code'].values

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [4]:
df2.set_index('Postal Code',inplace=True)

## Adding Coordinates

I tried with both google and geocoder, however could not manage to obtain coordinates for all the nieghborhoods. 

In [5]:
#!pip install geocoder
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab


In [6]:
from geopy.geocoders import Nominatim

latitudes=[]
longitudes=[]

for code in df2.index:
    # initialize your variable to None
    g=None
    
    print(code)
    # loop until you get the coordinates
    while(g is None):
        geolocator = Nominatim(user_agent="can_explorer")
        g = geolocator.geocode('{}, Toronto, Ontario'.format(code))
        print(g)
    latitudes.append(g.latitude)
    longitudes.append(g.longitude)
    
print('Done!')

M3A
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada
M4A
None
None
None
None
None
None


KeyboardInterrupt: 

I eventually loaded the file with coordinates and joined the tables. 

In [7]:
path='/home/IT043026/Desktop/git_projects/Geospatial_Coordinates.csv'
coord=pd.read_csv(path)
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
coord['Postal Code'].values
coord.set_index('Postal Code',inplace=True)

In [9]:
df3=pd.concat([df2,coord.reindex(df2.index)], axis=1)

In [10]:
df3.reset_index(inplace=True)
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Neighborhoods selection for K-means
Now I select only the borough that contains the word 'York'. Using coordinates of each row I will recall the venues of the neighborhoods. 

In [11]:
df4=df3[df3['Borough'].str.contains('York')]

In [12]:
df4.shape

(34, 5)

In [13]:
import folium
latitude=43.7001100
longitude=-79.4163000

In [14]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Add FourSquare Credentials and find nearby venues for all neighborhood selected

In [15]:
CLIENT_ID = '3GBYNQRL5Q1R44GR4C3IS32VX3IFLDNIQCC0YMJDL0BBKWKT' # your Foursquare ID
CLIENT_SECRET = 'XGYBBU1IPH0JVX3AISVXVWKVJ0PRACEQ054FB0YQIWRGITSE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3GBYNQRL5Q1R44GR4C3IS32VX3IFLDNIQCC0YMJDL0BBKWKT
CLIENT_SECRET:XGYBBU1IPH0JVX3AISVXVWKVJ0PRACEQ054FB0YQIWRGITSE


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], #) for v in results]) 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
Toronto_venues = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [18]:
print(Toronto_venues.shape)
Toronto_venues.head()

(340, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


I now count how many venues are nearby each neighborhood. Some of the numbers are really low.

In [266]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",5,5,5,5,5,5
Don Mills,27,27,27,27,27,27
Downsview,16,16,16,16,16,16
"East Toronto, Broadview North (Old East York)",4,4,4,4,4,4
"Fairview, Henry Farm, Oriole",67,67,67,67,67,67


In [19]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 121 uniques categories.


I hot encode the categories, so to group by neighborhood and list the top venues

In [44]:
# one hot encoding
tor_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
nei = tor_onehot['Neighborhood']
tor_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
tor_onehot.insert(0, 'Neighborhood', nei)

tor_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
tor_onehot.shape

(340, 122)

In [46]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.shape

(29, 122)

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

I select just 5 top venues, given the small numbers of before, indeed neighborhoods having only few venues will then list in alphabetical order.

In [55]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Supermarket,Middle Eastern Restaurant
1,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Yoga Studio
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Coffee Shop
3,Caledonia-Fairbanks,Park,Pool,Women's Store,Electronics Store,Comfort Food Restaurant
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Convenience Store,Restaurant,Sandwich Place,Bar,Discount Store


I finally run kmeans with 5 clusters to see the results, and plot the results on a map. 

In [49]:
#!pip install sklearn

In [56]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [57]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=100, max_iter=1000).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 4], dtype=int32)

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='inner')

tor_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Bus Stop,Food & Drink Shop,Yoga Studio,Electronics Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Pizza Place,Dog Run
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Miscellaneous Shop,Boutique,Coffee Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Japanese Restaurant,Asian Restaurant,Coffee Shop,Restaurant
13,M3C,North York,Don Mills,43.725900,-79.340923,0,Gym,Japanese Restaurant,Asian Restaurant,Coffee Shop,Restaurant


In [59]:
tor_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Bus Stop,Food & Drink Shop,Yoga Studio,Electronics Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Pizza Place,Dog Run
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Miscellaneous Shop,Boutique,Coffee Shop
7,M3B,North York,Don Mills,43.745906,-79.352188,0,Gym,Japanese Restaurant,Asian Restaurant,Coffee Shop,Restaurant
13,M3C,North York,Don Mills,43.725900,-79.340923,0,Gym,Japanese Restaurant,Asian Restaurant,Coffee Shop,Restaurant


In [60]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The result shows 3 populated clusters and 2 clusters composed by only one neighborhood.